In [2]:
import altair as alt
import pandas as pd

data = pd.DataFrame([dict(id=i) for i in range(1, 101)])

person = (
    "M1.7 -1.7h-0.8c0.3 -0.2 0.6 -0.5 0.6 -0.9c0 -0.6 "
    "-0.4 -1 -1 -1c-0.6 0 -1 0.4 -1 1c0 0.4 0.2 0.7 0.6 "
    "0.9h-0.8c-0.4 0 -0.7 0.3 -0.7 0.6v1.9c0 0.3 0.3 0.6 "
    "0.6 0.6h0.2c0 0 0 0.1 0 0.1v1.9c0 0.3 0.2 0.6 0.3 "
    "0.6h1.3c0.2 0 0.3 -0.3 0.3 -0.6v-1.8c0 0 0 -0.1 0 "
    "-0.1h0.2c0.3 0 0.6 -0.3 0.6 -0.6v-2c0.2 -0.3 -0.1 "
    "-0.6 -0.4 -0.6z"
)

alt.Chart(data).transform_calculate(
    row="ceil(datum.id/10)"
).transform_calculate(
    col="datum.id - datum.row*10"
).mark_point(
    filled=True,
    size=50
).encode(
    x=alt.X("col:O", axis=None),
    y=alt.Y("row:O", axis=None),
    shape=alt.ShapeValue(person)
).properties(
    width=400,
    height=400
).configure_view(
    strokeWidth=0
)

alt.Chart(...)

In [5]:
import altair as alt
import pandas as pd
import numpy as np

# 1. The "Human" Geometry (SVG Path)
person_icon = (
    "M1.7 -1.7h-0.8c0.3 -0.2 0.6 -0.5 0.6 -0.9c0 -0.6 "
    "-0.4 -1 -1 -1c-0.6 0 -1 0.4 -1 1c0 0.4 0.2 0.7 0.6 "
    "0.9h-0.8c-0.4 0 -0.7 0.3 -0.7 0.6v1.9c0 0.3 0.3 0.6 "
    "0.6 0.6h0.2c0 0 0 0.1 0 0.1v1.9c0 0.3 0.2 0.6 0.3 "
    "0.6h1.3c0.2 0 0.3 -0.3 0.3 -0.6v-1.8c0 0 0 -0.1 0 "
    "-0.1h0.2c0.3 0 0.6 -0.3 0.6 -0.6v-2c0.2 -0.3 -0.1 "
    "-0.6 -0.4 -0.6z"
)

# 2. Generate "ML Result" Data (100 People)
np.random.seed(42)
N = 100
data = pd.DataFrame({'id': range(N)})

conditions = [
    (data['id'] < 70),
    (data['id'] >= 70) & (data['id'] < 85),
    (data['id'] >= 85) & (data['id'] < 95),
    (data['id'] >= 95)
]
choices = ['Safe (True Neg)', 'Fraud Caught (True Pos)', 'False Alarm (False Pos)', 'Fraud Missed (False Neg)']

# FIX: Added default='Unknown' to prevent int/str type clash
data['status'] = np.select(conditions, choices, default='Unknown')

# Pre-calculate Grid positions
data['col'] = data['id'] % 10
data['row'] = data['id'] // 10

# 3. Build the Visualization
selection = alt.selection_point(fields=['status'], bind='legend')

base = alt.Chart(data).encode(
    x=alt.X('col:O', axis=None),
    y=alt.Y('row:O', axis=None, sort='descending')
).properties(
    title=dict(
        text="The Human Impact of Algorithmic Bias",
        subtitle=["Visualizing Model Performance on 100 Users.", "Click the Legend to isolate groups."],
        fontSize=16,
        subtitleFontSize=12
    ),
    width=500,
    height=500
)

points = base.mark_point(
    filled=True,
    size=100,
    opacity=1
).encode(
    shape=alt.ShapeValue(person_icon),
    color=alt.Color(
        'status:N', 
        scale=alt.Scale(
            domain=['Safe (True Neg)', 'Fraud Caught (True Pos)', 'False Alarm (False Pos)', 'Fraud Missed (False Neg)'],
            range=['#E0E0E0', '#2E86AB', '#D90429', '#EF233C']
        ),
        legend=alt.Legend(title="Model Decision", orient="right")
    ),
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1)),
    tooltip=['status', 'id']
).add_params(
    selection
)

points.configure_view(
    strokeWidth=0
).configure_title(
    anchor='start',
    offset=20
)

alt.Chart(...)

In [7]:
import altair as alt
import pandas as pd
import numpy as np

# --- 1. The Geometry (Person Icon) ---
person_icon = (
    "M1.7 -1.7h-0.8c0.3 -0.2 0.6 -0.5 0.6 -0.9c0 -0.6 "
    "-0.4 -1 -1 -1c-0.6 0 -1 0.4 -1 1c0 0.4 0.2 0.7 0.6 "
    "0.9h-0.8c-0.4 0 -0.7 0.3 -0.7 0.6v1.9c0 0.3 0.3 0.6 "
    "0.6 0.6h0.2c0 0 0 0.1 0 0.1v1.9c0 0.3 0.2 0.6 0.3 "
    "0.6h1.3c0.2 0 0.3 -0.3 0.3 -0.6v-1.8c0 0 0 -0.1 0 "
    "-0.1h0.2c0.3 0 0.6 -0.3 0.6 -0.6v-2c0.2 -0.3 -0.1 "
    "-0.6 -0.4 -0.6z"
)

# --- 2. The Data Simulation (Economics) ---
np.random.seed(99)
N = 100
data = pd.DataFrame({'id': range(N)})

# Simulate "Willingness To Pay" (WTP)
# We use a normal distribution to create a realistic "S" curve demand
data['wtp'] = np.random.normal(loc=50, scale=15, size=N).astype(int)
data = data.sort_values('wtp', ascending=False).reset_index(drop=True)

# Create Segments based on WTP (Price Elasticity Zones)
# High WTP = Inelastic (Will pay anything) -> Gold
# Mid WTP = Elastic (Sensitive) -> Cyan
# Low WTP = Highly Elastic (Cheap) -> Magenta
conditions = [
    (data['wtp'] > 60),
    (data['wtp'] <= 60) & (data['wtp'] > 40),
    (data['wtp'] <= 40)
]
choices = ['1. Inelastic (Loyalists)', '2. Elastic (Swing Market)', '3. Price Sensitive (Risk)']
colors = ['#FFD700', '#00F5FF', '#FF007F'] # Gold, Neon Cyan, Neon Magenta

data['segment'] = np.select(conditions, choices, default='Unknown')

# Grid Logic (Sorted by WTP so colors flow visually)
data['row'] = data.index // 10
data['col'] = data.index % 10

# --- 3. The Visualization ---

# A. The Selector (Interaction)
selection = alt.selection_point(fields=['segment'], bind='legend')

# B. LEFT PLOT: The Demand Manifold (The "Super Vis")
# We visualize the cumulative demand as a glowing area
demand_curve = alt.Chart(data).mark_area(
    interpolate='monotone',
    fillOpacity=0.6,
    line={'color':'white'}
).encode(
    x=alt.X('id:Q', title='Market Volume (Cumulative)', axis=alt.Axis(labels=False, grid=False)),
    y=alt.Y('wtp:Q', title='Willingness to Pay ($)', scale=alt.Scale(domain=[0, 100])),
    color=alt.Color('segment:N', scale=alt.Scale(range=colors), legend=None),
    tooltip=['segment', 'wtp']
).properties(
    width=300,
    height=400,
    title="The Demand Manifold"
).add_params(
    selection
).transform_filter(
    selection
)

# Add a "Price Floor" line to make it look technical
# (Shows the theoretical market clearing price)
rule = alt.Chart(pd.DataFrame({'y': [50]})).mark_rule(
    color='white', strokeDash=[4, 4], size=2
).encode(y='y')

left_chart = (demand_curve + rule)

# C. RIGHT PLOT: The Human Grid (ISOTYPE)
# Sorted by WTP to match the flow of the left chart
people_grid = alt.Chart(data).mark_point(
    filled=True,
    size=100, # User requested size
    opacity=1
).encode(
    x=alt.X('col:O', axis=None),
    y=alt.Y('row:O', axis=None), # No sort needed as data is pre-sorted
    shape=alt.ShapeValue(person_icon),
    color=alt.Color('segment:N', 
                    scale=alt.Scale(domain=choices, range=colors),
                    legend=alt.Legend(title="Elasticity Zone", orient="bottom", titleFontSize=12)),
    tooltip=['id', 'wtp', 'segment'],
    opacity=alt.condition(selection, alt.value(1), alt.value(0.1))
).properties(
    width=400,
    height=400,
    title="The Consumer Population"
).add_params(
    selection
)

# --- 4. Composition ---
dashboard = alt.hconcat(
    left_chart, 
    people_grid
).resolve_scale(
    color='shared'
).configure_view(
    strokeWidth=0
).configure(
    background='#1e1e1e', # Dark Mode Background
    title={'color': 'white', 'fontSize': 18},
    legend={'labelColor': 'white', 'titleColor': 'white'},
    axis={'labelColor': 'white', 'titleColor': 'white', 'gridColor': '#333'}
)

dashboard

/Users/pstaif/Downloads/MyApps/econ_visual/venv/lib/python3.13/site-packages/IPython/core/interactiveshell.py:3699: UserWarning: Automatically deduplicated selection parameter with identical configuration. If you want independent parameters, explicitly name them differently (e.g., name='param1', name='param2'). See https://github.com/vega/altair/issues/3891
  exec(code_obj, self.user_global_ns, self.user_ns)


alt.HConcatChart(...)

In [11]:
import altair as alt
import pandas as pd
import numpy as np

# --- 1. The Geometry (The "Ghost" capable icon) ---
person_icon = (
    "M1.7 -1.7h-0.8c0.3 -0.2 0.6 -0.5 0.6 -0.9c0 -0.6 "
    "-0.4 -1 -1 -1c-0.6 0 -1 0.4 -1 1c0 0.4 0.2 0.7 0.6 "
    "0.9h-0.8c-0.4 0 -0.7 0.3 -0.7 0.6v1.9c0 0.3 0.3 0.6 "
    "0.6 0.6h0.2c0 0 0 0.1 0 0.1v1.9c0 0.3 0.2 0.6 0.3 "
    "0.6h1.3c0.2 0 0.3 -0.3 0.3 -0.6v-1.8c0 0 0 -0.1 0 "
    "-0.1h0.2c0.3 0 0.6 -0.3 0.6 -0.6v-2c0.2 -0.3 -0.1 "
    "-0.6 -0.4 -0.6z"
)

# --- 2. Data Simulation ---
np.random.seed(42)
N = 100

# Market A: Necessity (Inelastic)
df_inelastic = pd.DataFrame({'id': range(N)})
df_inelastic['market'] = 'Market A: Necessity (Inelastic)'
df_inelastic['wtp'] = np.random.normal(loc=80, scale=10, size=N).astype(int)

# Market B: Luxury (Elastic)
df_elastic = pd.DataFrame({'id': range(N)})
df_elastic['market'] = 'Market B: Luxury (Elastic)'
df_elastic['wtp'] = np.random.normal(loc=40, scale=15, size=N).astype(int)

data = pd.concat([df_inelastic, df_elastic])
data['col'] = data.groupby('market').cumcount() % 10
data['row'] = data.groupby('market').cumcount() // 10

# --- 3. The Interactive Logic (FIXED) ---
# We give the param a specific internal 'name' ("price_cutoff")
# This allows us to refer to it in the condition string below.
price_slider = alt.param(
    value=30,
    bind=alt.binding_range(min=0, max=100, step=5, name='Current Price ($) '),
    name='price_cutoff' 
)

# --- 4. The Visualization ---
base = alt.Chart(data).encode(
    x=alt.X('col:O', axis=None),
    y=alt.Y('row:O', axis=None, sort='descending')
).properties(
    width=300,
    height=300
)

chart = base.mark_point(
    filled=True,
    size=50
).encode(
    shape=alt.ShapeValue(person_icon),
    
    # Use the parameter name 'price_cutoff' directly in the string
    color=alt.condition(
        "datum.wtp >= price_cutoff", 
        alt.Color('market:N', scale=alt.Scale(range=['#00FF00', '#00BFFF']), legend=None),
        alt.value('#333333') # Ghost color
    ),
    
    opacity=alt.condition(
        "datum.wtp >= price_cutoff",
        alt.value(1),
        alt.value(0.2)
    ),
    
    tooltip=['wtp', 'market']
).add_params(
    price_slider
).facet(
    column=alt.Column('market:N', title=None, header=alt.Header(
        labelColor='white', 
        labelFontSize=20, 
        labelFontWeight='bold'
    ))
).resolve_scale(
    color='independent'
)

# --- 5. Cyberpunk Styling ---
chart.configure(
    background='#111111',
    view=dict(strokeWidth=0)
).configure_title(
    color='white'
)

alt.FacetChart(...)

In [12]:
import altair as alt
import pandas as pd
import numpy as np

# --- 1. The Geometry ---
person_icon = (
    "M1.7 -1.7h-0.8c0.3 -0.2 0.6 -0.5 0.6 -0.9c0 -0.6 "
    "-0.4 -1 -1 -1c-0.6 0 -1 0.4 -1 1c0 0.4 0.2 0.7 0.6 "
    "0.9h-0.8c-0.4 0 -0.7 0.3 -0.7 0.6v1.9c0 0.3 0.3 0.6 "
    "0.6 0.6h0.2c0 0 0 0.1 0 0.1v1.9c0 0.3 0.2 0.6 0.3 "
    "0.6h1.3c0.2 0 0.3 -0.3 0.3 -0.6v-1.8c0 0 0 -0.1 0 "
    "-0.1h0.2c0.3 0 0.6 -0.3 0.6 -0.6v-2c0.2 -0.3 -0.1 "
    "-0.6 -0.4 -0.6z"
)

# --- 2. Data Generation ---
np.random.seed(42)
N = 100

# Market A: Inelastic (Medicine)
df_a = pd.DataFrame({'id': range(N)})
df_a['market'] = 'A: Inelastic (Medicine)'
df_a['wtp'] = np.random.normal(loc=80, scale=10, size=N).astype(int)

# Market B: Elastic (Cinema)
df_b = pd.DataFrame({'id': range(N)})
df_b['market'] = 'B: Elastic (Cinema)'
df_b['wtp'] = np.random.normal(loc=40, scale=15, size=N).astype(int)

# Combine for People Plots
people_data = pd.concat([df_a, df_b])
people_data['col'] = people_data.groupby('market').cumcount() % 10
people_data['row'] = people_data.groupby('market').cumcount() // 10

# Generate Aggregate Curve Data (Price vs Quantity)
# We calculate: For every price P, how many people have WTP >= P?
curve_rows = []
for m_name, df_subset in [('A: Inelastic (Medicine)', df_a), ('B: Elastic (Cinema)', df_b)]:
    for p in range(0, 120): # Price range 0 to 120
        quantity = len(df_subset[df_subset['wtp'] >= p])
        curve_rows.append({'market': m_name, 'price': p, 'quantity': quantity})

curve_data = pd.DataFrame(curve_rows)

# --- 3. The Shared Controller ---
# This slider controls EVERYTHING
price_slider = alt.param(
    value=30,
    bind=alt.binding_range(min=0, max=100, step=1, name='Set Market Price ($) '),
    name='price_cutoff'
)

# --- 4. The Top Row: Demand Curves (Macro) ---

# We split this because we want distinct colors for A and B
# Chart A (Green)
line_a = alt.Chart(curve_data[curve_data['market'].str.contains('A')]).mark_area(
    interpolate='monotone', fillOpacity=0.3, line=True, color='#00FF00'
).encode(
    x=alt.X('quantity:Q', title='Quantity Demanded', scale=alt.Scale(domain=[0, 100])),
    y=alt.Y('price:Q', title='Price ($)', scale=alt.Scale(domain=[0, 100]))
).properties(title="Demand Curve A", height=200, width=250)

# Chart B (Blue)
line_b = alt.Chart(curve_data[curve_data['market'].str.contains('B')]).mark_area(
    interpolate='monotone', fillOpacity=0.3, line=True, color='#00BFFF'
).encode(
    x=alt.X('quantity:Q', title='Quantity Demanded', scale=alt.Scale(domain=[0, 100])),
    y=alt.Y('price:Q', title=None, scale=alt.Scale(domain=[0, 100]))
).properties(title="Demand Curve B", height=200, width=250)

# The "Laser Level" (Price Line)
# This uses the slider param to draw a horizontal rule
price_rule = alt.Chart(pd.DataFrame({'dummy': [1]})).mark_rule(
    color='white', strokeWidth=2, strokeDash=[4,4]
).encode(
    y=alt.Y('price_cutoff:Q') # Links to the param name
)

top_row = (line_a + price_rule) | (line_b + price_rule)

# --- 5. The Bottom Row: The People (Micro) ---

base_people = alt.Chart(people_data).encode(
    x=alt.X('col:O', axis=None),
    y=alt.Y('row:O', axis=None, sort='descending')
).properties(
    width=250,
    height=250
)

# We facet this manually to ensure colors align with the top row
# People A (Green)
people_a = base_people.transform_filter(
    alt.datum.market == 'A: Inelastic (Medicine)'
).mark_point(filled=True, size=400).encode(
    shape=alt.ShapeValue(person_icon),
    color=alt.condition("datum.wtp >= price_cutoff", alt.value('#00FF00'), alt.value('#333')),
    opacity=alt.condition("datum.wtp >= price_cutoff", alt.value(1), alt.value(0.2)),
    tooltip=['wtp']
).properties(title="Market A Population")

# People B (Blue)
people_b = base_people.transform_filter(
    alt.datum.market == 'B: Elastic (Cinema)'
).mark_point(filled=True, size=400).encode(
    shape=alt.ShapeValue(person_icon),
    color=alt.condition("datum.wtp >= price_cutoff", alt.value('#00BFFF'), alt.value('#333')),
    opacity=alt.condition("datum.wtp >= price_cutoff", alt.value(1), alt.value(0.2)),
    tooltip=['wtp']
) .properties(title="Market B Population")

bottom_row = people_a | people_b

# --- 6. Composition ---
dashboard = alt.vconcat(
    top_row,
    bottom_row
).add_params(
    price_slider
).configure(
    background='#111111',
    view=dict(strokeWidth=0)
).configure_axis(
    labelColor='white', titleColor='white', gridColor='#333'
).configure_title(
    color='white', fontSize=16, anchor='start'
)

dashboard

alt.VConcatChart(...)